In [1]:
!pip install boto3

     |################################| 131 kB 11.9 MB/s eta 0:00:01
     |################################| 7.6 MB 13.9 MB/s eta 0:00:01
     |################################| 79 kB 15.3 MB/s eta 0:00:01


In [3]:
from torch.utils.data import Dataset
from torchvision import transforms
import boto3

In [9]:
bucket_name = 'wiai-pest-interns-mumbai-2021'

In [11]:
s3 = boto3.resource('s3')
bucket = s3.Bucket(bucket_name)

In [14]:
l = [obj.key for obj in bucket.objects.all()]

NoCredentialsError: Unable to locate credentials

In [5]:
class ImageDataset(Dataset):
    def __init__(self, bucket_name, path='./images', transform=None):
        self.path = path
        self.s3 = boto3.resource('s3')
        self.bucket = self.s3.Bucket(path)
        self.files = [obj.key for obj in self.bucket.objects.all()]
        self.transform = transform
        if transform is None:
            self.transform = transforms.Compose([
                transforms.Resize((128, 128)),
                transforms.ToTensor(),
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
            ])

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        img_name = self.files[idx]

        # we need to download the file from S3 to a temporary file locally
        # we need to create the local file name
        obj = self.bucket.Object(img_name)
        tmp = tempfile.NamedTemporaryFile()
        tmp_name = '{}.jpg'.format(tmp.name)

        # now we can actually download from S3 to a local place
        with open(tmp_name, 'wb') as f:
            obj.download_fileobj(f)
            f.flush()
            f.close()
            image = Image.open(tmp_name)

        if self.transform:
            image = self.transform(image)

        return image